In [1]:
from qiskit.circuit.library import QFT
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from qiskit import transpile
from QASMBench.interface.qiskit import QASMBenchmark
from disqco.graphs.quantum_network import QuantumNetwork


num_qubits = 256

circuit = cp_fraction(num_qubits, num_qubits, fraction=0.5)

# circuit = QFT(num_qubits, do_swaps=False)

num_partitions = 4  # Define the number of partitions or QPUs you will distribute over
depth = circuit.depth()

qpu_size = num_qubits // num_partitions + 1 # Define the number of qubits per QPU. For simplicity, we divide qubits evenly and add one additional space to each for teleportation (an extra space is needed for the teleportation qubit so the algorithmr requires it!)
qpu_sizes = [qpu_size] * num_partitions # Store the number of qubits per QPU in a list

# Create a quantum network with the specified number of qubits and partitions

# If we do not specificy the connectivity, we have all-to-all by default.

quantum_network = QuantumNetwork(qpu_sizes)



basis_gates = ['u', 'cp']

# Transpile the circuit to the basis gates
circuit = transpile(circuit, basis_gates=basis_gates)

print(f'Number of qubits in circuit {circuit.num_qubits}')
print(f'Circuit depth: {circuit.depth()}')

Number of qubits in circuit 256
Circuit depth: 256


In [2]:
from disqco.parti.FM.fiduccia import *

FM_partitioner = FiducciaMattheyses(circuit=circuit,
                                    network=quantum_network)

Number of layers: 258


In [3]:
results = FM_partitioner.partition()

graph = FM_partitioner.hypergraph

print(f'Best cost: {results['best_cost']}')


if len(graph.nodes) < 500:
    %load_ext jupyter_tikz
    graph.draw(assignment=results['best_assignment'], qpu_info=qpu_sizes)





KeyboardInterrupt: 

In [79]:
# from disqco.parti.genetic.genetic_algorithm_beta import *

# GA_partitioner = GeneticPartitioner(circuit=circuit, network=quantum_network, group_gates=True, log=True, multi_process=True)

# results = GA_partitioner.partition()
# print(f'Best cost: {results['best_cost']}')

In [4]:
from disqco.graphs.coarsening.coarsener import HypergraphCoarsener

coarsener = HypergraphCoarsener()

coarseneing_method = coarsener.coarsen_recursive_batches_mapped

results = FM_partitioner.multilevel_partition(coarsener=coarseneing_method)

Number of layers: 258
Number of layers: 258
Number of layers: 258
Number of layers: 258
Number of layers: 258
Number of layers: 258
Number of layers: 258
Number of layers: 258
Best cost at level 0: 8615
Best cost at level 1: 8175
Best cost at level 2: 7756
Best cost at level 3: 7402
Best cost at level 4: 7024
Best cost at level 5: 6723
Best cost at level 6: 6572
Best cost at level 7: 6501
Best cost at level 8: 6474


In [91]:
print(f'Best cost: {results['best_cost']}')

print(f'Best assignment: {results["best_assignment"]}')

Best cost: 6494
Best assignment: [[1 0 1 ... 3 2 1]
 [1 0 1 ... 3 2 1]
 [1 0 1 ... 3 2 1]
 ...
 [3 2 3 ... 3 2 3]
 [3 2 3 ... 3 2 1]
 [3 2 3 ... 3 2 1]]


In [82]:
# results = GA_partitioner.multilevel_partition(coarsener=coarseneing_method)

In [83]:
# from disqco.graphs.quantum_network import linear_coupling

# coupling = linear_coupling(num_partitions)

# network = QuantumNetwork(qpu_sizes, qpu_connectivity=coupling)
# FM_partitioner = FiducciaMattheyses(circuit=circuit,
#                                     network=network)


# results = FM_partitioner.partition(log=True)

In [84]:
# results = FM_partitioner.multilevel_partition()

In [ ]:
from disqco.parti.FM.fiduccia_ext import *
from disqco.parti.FM.FM_methods_ext import *

best_assignment = results["best_assignment"]

initial_qubit_assignment = best_assignment.transpose()

print(initial_qubit_assignment)

FM_partitioner = FiducciaMattheysesExt(circuit=circuit, network=quantum_network, initial_qubit_assignment=initial_qubit_assignment)


results = FM_partitioner.partition(passes=10, stochastic=False, log=True)




[[1 1 1 ... 3 3 3]
 [0 0 0 ... 2 2 2]
 [1 1 1 ... 3 3 3]
 ...
 [3 3 3 ... 3 3 3]
 [2 2 2 ... 2 2 2]
 [1 1 1 ... 3 1 1]]
Number of layers: 258
Initial cost: 8368.0


In [86]:
graph = FM_partitioner.hypergraph

graph.draw(qubit_assignment=results['best_qubit_assignment'], gate_assignment=results['best_gate_assignment'],  qpu_info=qpu_sizes,show_labels=False)

KeyboardInterrupt: 